## IMPORTS 

In [30]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
from sklearn.metrics import confusion_matrix

In [31]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [32]:
#Check TensorFlow Version
tf.__version__

'1.13.1'

In [33]:
#Check Keras Version
tf.keras.__version__

'2.2.4-tf'

## IMPORT TWEET TEXT DATA


In [34]:
#Import
dem_train_text1 = pd.read_csv('../data_master/nlp_feed_data/dem_tweets.csv')                        

rep_test_text1 = pd.read_csv('../data_master/nlp_feed_data/rep_tweets.csv')

#Drop and show df
dem_train_text2 = dem_train_text1.drop(['Unnamed: 0'], axis=1)
print('DEMOCRAT TRAIN TWEETS')
print(dem_train_text2.head())
print(type(dem_train_text2)) 
      
rep_test_text2 = rep_test_text1.drop(['Unnamed: 0'], axis=1)
print('REPUBLICAN TEST TWEETS')
print(rep_test_text2.head())
print(type(rep_test_text2))


DEMOCRAT TRAIN TWEETS
                                               tweet
0  We're in a battle for the soul of this nation....
1  Today marks 50 years since the Stonewall upris...
2  I’ve fought my heart out to ensure that civil ...
3  At last night’s first Democratic debate, I lai...
4  I just got off the debate stage, where I laid ...
<class 'pandas.core.frame.DataFrame'>
REPUBLICAN TEST TWEETS
                                               tweet
0  NO American should worry about the cost of lif...
1  ALL American families deserve to live in safe ...
2  A targeted attack on our democracy is no laugh...
3  House Democrats are FOCUSED on protecting our ...
4  Mental health care is health care. We must pro...
<class 'pandas.core.frame.DataFrame'>


## CONVERT DATAFRAMES TO STRING FOR PREP TO FEED TO NLP

In [35]:
#Dataframe to list then list to strings
rep_test_text = rep_test_text.values.tolist()
for i in range(len(rep_test_text)):
    rep_test_text[i] = str(rep_test_text[i])
dem_train_text = dem_train_text.values.tolist()
for i in range(len(dem_train_text)):
    dem_train_text[i] = str(dem_train_text[i])
    
#Show strings
print('REPUBLICAN TEST LIST LENGTH')
print(len([rep_test_text]))
print(type(rep_test_text))
print('DEMOCRAT TRAIN LIST LENGTH')
print(len([dem_train_text]))
print(type(dem_train_text))

NameError: name 'rep_test_text' is not defined

## (NOT NEEDED BUT WANT TO KEEP IN CASE OF PIVOT)

In [36]:
IMPORT CLASSIFIER DATA

#Import
#dem_train_y = pd.read_csv('../data_master/nlp_feed_data/dem_classifier.csv')                        

#rep_train_x = pd.read_csv('../data_master/nlp_feed_data/rep_classifier.csv')

#Drop and show
#dem_train_y = dem_train_y.drop(['Unnamed: 0'], axis=1)
#print('DEMOCRAT CLASSIFIER')
#print(dem_train_y.head())
#print(type(dem_train_y))
      
#rep_train_x = rep_train_x.drop(['Unnamed: 0'], axis=1)
#print('REPUBLICAN CLASSIFIER')
#print(rep_train_x.head())
#print(type(rep_train_x))

CONVERT DATAFRAMES TO LIST FOR PREP FEED TO NLP

#rep_train_x = rep_train_x.values.tolist()
#for i in range(len(rep_train_x)):
#    rep_train_x[i] = str(rep_train_x[i])
#dem_train_y = dem_train_y.values.tolist()

#print('REPUBLICAN LIST LENGTH')
#print(len([rep_train_x]))
#print(type(rep_train_x))
#print('DEMOCRAT LIST LENGTH')
#print(len([dem_train_y]))
#print(type(dem_train_y))

SyntaxError: invalid syntax (<ipython-input-36-feb4758469ec>, line 1)

## CREATE DATAFRAMES WITH CLASSIFIER VALUE 1 OR 0

In [37]:
dem_train_y = [i for i in np.arange(1,9055,1)]
print('Interger Return off of DF initial build (7th posistion)')
print(dem_train_y[7])
for n, i in enumerate(dem_train_y):
              dem_train_y[n] = 1
print('Interger Return off of DF refactored (0) build')
print(dem_train_y[7])
dem_train_y

#list to DF 
dem_train_y_df = pd.DataFrame(dem_train_y)


rep_test_x = [i for i in np.arange(1,14449,1)]
print('Interger Return off of DF initial build (7th posistion)')
print(rep_test_x[7])
for n, i in enumerate(rep_test_x):
              rep_test_x[n] = 0
print('Interger Return off of DF refactored (1) build')
print(type(rep_test_x))
#list to DF 
rep_test_x_df = pd.DataFrame(rep_test_x)
rep_test_text2
rep_test_x_df

Interger Return off of DF initial build (7th posistion)
8
Interger Return off of DF refactored (0) build
1
Interger Return off of DF initial build (7th posistion)
8
Interger Return off of DF refactored (1) build
<class 'list'>


,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


## DATA REWORK

In [38]:
#rep_tweet_0 = pd.concat([rep_test_text2, rep_test_x_df], axis=1, join='union')
#dem_tweet_1 = pd.concat([dem_train_text2, dem_train_y_df], axis=1, join='union')
#merged = pd.merge(rep_test_text2, rep_test_x_df, on=['tweet','0'])
rep_tc_0 = pd.merge(rep_test_text2, rep_test_x_df, left_index=True, right_index=True)
dem_tc_1 = pd.merge(dem_train_text2, dem_train_y_df, left_index=True, right_index=True)
dem_tc_1.head()


,tweet,0
0,We're in a battle for the soul of this nation....,1
1,Today marks 50 years since the Stonewall upris...,1
2,I’ve fought my heart out to ensure that civil ...,1
3,"At last night’s first Democratic debate, I lai...",1
4,"I just got off the debate stage, where I laid ...",1


In [39]:
dem_rep_all = dem_tc_1.append(rep_tc_0, ignore_index=True)
dem_rep_all.head()


,tweet,0
0,We're in a battle for the soul of this nation....,1
1,Today marks 50 years since the Stonewall upris...,1
2,I’ve fought my heart out to ensure that civil ...,1
3,"At last night’s first Democratic debate, I lai...",1
4,"I just got off the debate stage, where I laid ...",1


In [40]:
dem_rep_allv2 = dem_rep_all.drop("tweet", axis=1)
dem_rep_allv2.head()

,0
0,1
1,1
2,1
3,1
4,1


In [41]:
dem_rep_allv2 = dem_rep_allv2.rename(columns={0: 1})
dem_rep_allv2

,1
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [42]:
dem_rep_allv7 = dem_rep_allv2.rename(columns={1: 2})
dem_rep_allv7

,2
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [43]:
dem_rep_allv3 = pd.merge(dem_rep_all, dem_rep_allv2, left_index=True, right_index=True)
dem_rep_allv5 = pd.merge(dem_rep_allv3, dem_rep_allv7, left_index=True, right_index=True)
dem_rep_allv5
dem_rep_allv2

,1
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [44]:
X = dem_rep_all.drop("tweet", axis=1)
y = dem_rep_all["tweet"]
print(X.shape, y.shape)

(23502, 1) (23502,)


In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [46]:
print(type(y_test))

<class 'pandas.core.series.Series'>


In [47]:
y_train = y_train.values.tolist()
for i in range(len(y_train)):
    y_train[i] = str(y_train[i])
y_test = y_test.values.tolist()
for i in range(len(y_test)):
    y_test[i] = str(y_test[i])

## COMBINE DATA SETS FOR USE LATER

In [48]:
data_text = y_train + y_test
data_text

['Our present-day crisis of mass incarceration has become a tool of voter suppression. Today, over 4.5 million Americans — disproportionately people of color — have lost their right to vote because they have served time in jail or prison for a felony conviction. 4/',
 '😒 https://t.co/iQ9cACHtnW',
 '#MaduroRegime has fired into territory of #Colombia. Receiving reports of injuries after this attack on sovereign Colombian territory. \n\nThe United States WILL help Colombia confront any aggression against them. https://t.co/lcF1uB9NB4',
 'Congress should pass my bill to #EndCorruptionNow – so the public can review the tax returns of federal candidates and ensure they’re running to serve the public interest, not their own financial interests. https://t.co/IJzMYJFLnU',
 "VP Biden on Mitt Romney's double standard: http://t.co/EIl3FgRk",
 '“Who do you trust to stand up for the middle class? @BarackObama is the man you should trust.”—VP Biden in Ocala, FL: http://t.co/ltsspSpo',
 'The message 

## TOKENIZER

In [49]:
num_words = 10000

In [50]:
tokenizer = Tokenizer(num_words=num_words)

In [51]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 836 ms, sys: 8.68 ms, total: 845 ms
Wall time: 843 ms


In [52]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [53]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'co': 3,
 't': 4,
 'https': 5,
 'of': 6,
 'a': 7,
 'in': 8,
 'and': 9,
 'for': 10,
 'is': 11,
 'amp': 12,
 'on': 13,
 'we': 14,
 'that': 15,
 'our': 16,
 'with': 17,
 'this': 18,
 'are': 19,
 'will': 20,
 'it': 21,
 'i': 22,
 'be': 23,
 'you': 24,
 'have': 25,
 'they': 26,
 'not': 27,
 'from': 28,
 'by': 29,
 'at': 30,
 'as': 31,
 'their': 32,
 'has': 33,
 'people': 34,
 'but': 35,
 'all': 36,
 'who': 37,
 'he': 38,
 'more': 39,
 'about': 40,
 'an': 41,
 'his': 42,
 'was': 43,
 'today': 44,
 'my': 45,
 'us': 46,
 '”': 47,
 'now': 48,
 'no': 49,
 'your': 50,
 'or': 51,
 'if': 52,
 's': 53,
 'up': 54,
 'do': 55,
 'u': 56,
 'president': 57,
 'just': 58,
 'great': 59,
 'out': 60,
 'must': 61,
 'them': 62,
 'one': 63,
 '–': 64,
 'can': 65,
 'country': 66,
 'what': 67,
 'so': 68,
 'time': 69,
 'american': 70,
 'should': 71,
 'americans': 72,
 'make': 73,
 'venezuela': 74,
 'trump': 75,
 'get': 76,
 'new': 77,
 'tax': 78,
 'need': 79,
 'would': 80,
 'health': 81,
 'than'

In [56]:
#Use the tokenizer to convert all democrat tweets in the training-set to lists of these tokens.
y_train_tokens = tokenizer.texts_to_sequences(y_train)

In [57]:
np.array(y_train_tokens[1])

array([5, 4, 3])

In [58]:
#Also need to convert the tweets in the test-set to tokens.
y_test_tokens = tokenizer.texts_to_sequences(y_test)

## PADDING AND TRUNCATING DATA

In [59]:
#Count the number of tokens in all the sequences in the data-set.
num_tokens = [len(tokens) for tokens in y_train_tokens + y_test_tokens]
num_tokens = np.array(num_tokens)


In [60]:
#Average number of tokens in a sequence
np.mean(num_tokens)

30.14773210790571

In [61]:
#Maximum number of tokens in a sequence
np.max(num_tokens)

61

In [62]:
#Max number of tokens we will allow is set to the average plus 2 standard deviations
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

55

In [63]:
#Covers about 99% of the data-set
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9939579610245937

## NVM ACTUAL PADDING HERE

In [65]:
pad = 'pre'

In [66]:
x_train_pad = pad_sequences(y_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [67]:
x_test_pad = pad_sequences(y_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [68]:
#Matrix for the train-set
x_train_pad.shape

(17626, 55)

In [69]:
#Matrix for the test-set
#
# !!! EXAMPLE HAS TEST AND TRAIN SHAPE THE EXACT SAME !!!
#
x_test_pad.shape

(5876, 55)

In [70]:
#Squence of tokens
np.array(y_test_tokens[1])

array([ 137,   21,   11,   17, 5164, 3950,   95,    4,    3])

In [71]:
#Squence of tokens
np.array(y_train_tokens[1])

array([5, 4, 3])

In [72]:
#Padded to create the following sequence
#
# !!! LOOKS DIFFERENT FROM THE EXAMPLE !!!
#
y_test_tokens[1]

[137, 21, 11, 17, 5164, 3950, 95, 4, 3]

## TOKENIZER INVERSE MAP

In [73]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [74]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [75]:
tokens_to_string(y_train_tokens[1])

'https t co'

In [76]:
tokens_to_string(y_test_tokens[1])

'here it is with link fixed http t co'

## CREATE THE RECURRENT NEURAL NETWORK

In [77]:
model = Sequential()

In [78]:
embedding_size = 8

In [79]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

Instructions for updating:
Colocations handled automatically by placer.


In [80]:
model.add(GRU(units=16, return_sequences=True))

In [81]:
model.add(GRU(units=8, return_sequences=True))

In [82]:
model.add(GRU(units=4))

In [83]:
model.add(Dense(1, activation='sigmoid'))

In [84]:
optimizer = Adam(lr=1e-3)

In [85]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [86]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 55, 8)             80000     
_________________________________________________________________
gru (GRU)                    (None, 55, 16)            1200      
_________________________________________________________________
gru_1 (GRU)                  (None, 55, 8)             600       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


## TRAIN THE RECURRENT NERUAL NETWORK

In [87]:
%%time
model.fit(x_train_pad, X_train,
          validation_split=0.05, epochs=3, batch_size=64)

Train on 16744 samples, validate on 882 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
16744/16744 [==============================] - 27s 2ms/sample - loss: 0.5163 - acc: 0.7370 - val_loss: 0.3516 - val_acc: 0.8537
Epoch 2/3
16744/16744 [==============================] - 26s 2ms/sample - loss: 0.2943 - acc: 0.8823 - val_loss: 0.3484 - val_acc: 0.8549
Epoch 3/3
16744/16744 [==============================] - 27s 2ms/sample - loss: 0.2319 - acc: 0.9101 - val_loss: 0.3446 - val_acc: 0.8628
CPU times: user 3min 15s, sys: 1min 18s, total: 4min 33s
Wall time: 1min 23s


## PERFORMANCE ON THE TEST SET


In [88]:
%%time
result = model.evaluate(x_test_pad, X_test)

5876/5876 [==============================] - 5s 813us/sample - loss: 0.3436 - acc: 0.8502
CPU times: user 11 s, sys: 5.4 s, total: 16.4 s
Wall time: 4.78 s


In [89]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 85.02%


In [101]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

In [102]:
tokens = tokenizer.texts_to_sequences(texts)

In [103]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

(8, 56)

In [104]:
model.predict(tokens_pad)

array([[0.9735117 ],
       [0.97351193],
       [0.9735116 ],
       [0.9735123 ],
       [0.97351044],
       [0.9735115 ],
       [0.97351193],
       [0.97350955]], dtype=float32)